In [1]:
import torch
import torch.nn as nn

In [69]:
a = torch.randn([5,1,12,100,100])

In [87]:
conv1 = nn.Conv3d(in_channels=1,out_channels=64,kernel_size=5,stride=1,padding=2)
conv2 = nn.Conv3d(in_channels=64,out_channels=128,kernel_size=3,stride=(1,2,2),padding=1)
conv3 = nn.Conv3d(in_channels=128,out_channels=128,kernel_size=3,stride=(2,2,2),padding=1)

deconv1 = nn.ConvTranspose3d(in_channels=128,out_channels=128,kernel_size=2,3,3,stride=(2,2,2),padding=1,output_padding=1)
deconv2 = nn.ConvTranspose3d(in_channels=128,out_channels=64,kernel_size=(3,4,4),stride=(1,2,2),padding=1,output_padding=0)
deconv3 = nn.ConvTranspose3d(in_channels=64,out_channels=1,kernel_size=5,stride=1,padding=2,output_padding=0)

In [86]:
deconv3(deconv2(deconv1(conv3(conv2(conv1(a)))))).squeeze().shape

RuntimeError: invalid argument 15: output padding must be smaller than either stride or dilation, but got aT: 1 aH: 1 aW: 1 dT: 1 dH: 2 dW: 2 dilationT: 1 dilationH: 1 dilationW: 1 at /opt/conda/conda-bld/pytorch_1533672544752/work/aten/src/THNN/generic/VolumetricFullDilatedConvolution.c:107

In [84]:
conv3(conv2(conv1(a))).squeeze().shape

torch.Size([5, 128, 6, 25, 25])